# DAC分组问题

In [1]:
import sys
sys.path.append("src")
import os
import json

import libpybiofeature

import utils
work_Dir = utils.workdir.workdir(os.getcwd(), 4)

import numpy as np
import pandas as pd

In [2]:
from Bio import SeqIO
def load_DAC_feature(TxSE_args: dict):

    # DAC
    DAC_feature = {
        "name": "DAC",
        "p": libpybiofeature.featurebuilder.build_dac_feature(
            path_to_fasta=TxSE_args['fasta']['p'],
            seq_id_list=[ seq.id for seq in SeqIO.parse(TxSE_args['fasta']['p'], "fasta") ],
            desc='p'
        ),
        "n": libpybiofeature.featurebuilder.build_dac_feature(
            path_to_fasta=TxSE_args['fasta']['n'],
            seq_id_list=[ seq.id for seq in SeqIO.parse(TxSE_args['fasta']['n'], "fasta") ],
            desc='n'
        ),
    }

    print(DAC_feature['n'].shape[0], DAC_feature['p'].shape[0])

    return DAC_feature

In [3]:
prot_type = 6
cter_bool = False
Tx_arg = {
    "type": f'T{prot_type}',
    'fasta': {
        'cter': cter_bool,
        'p': "data/T6SE/anti-bacterial-effector_p.fasta",
        'n': "data/T6SE/anti-bacterial-effector_n.fasta"
    },
}
save_dir = "out/libfeatureselection/Six_feature_research/dac/ab"

In [4]:
aac_data = load_DAC_feature(
    TxSE_args=Tx_arg
)
aa_type = list(aac_data['p'].columns)

n_DAC: 100%|██████████| 53/53 [00:00<00:00, 10384.37it/s]

53 53


In [5]:
import matplotlib as mpl

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.use14corefonts'] = False
# mpl.rcParams['pdf.usecorefonts'] = True
mpl.rcParams['pdf.compression'] = 9

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'nature'])

from matplotlib.ticker import MaxNLocator

import seaborn as sns

In [6]:
aac_data_transformed = pd.concat([aac_data['p'], aac_data['n']], keys=['T6SP-AB', 'non-T6SP']).reset_index(level=0).rename({
    "level_0": "Type"
}, axis=1, inplace=False).melt(id_vars="Type", value_vars=aa_type, ignore_index=False).rename({
    "variable": "Amino acid",
    "value": "DAC"
}, axis=1, inplace=False)

In [7]:
aac_data_transformed

,Type,Amino acid,DAC
WP_005299972.1,T6SP-AB,"A,A",0.011017
NP_248951.1,T6SP-AB,"A,A",0.008403
pdb|6H56|A,T6SP-AB,"A,A",0.015873
PTC37822.1,T6SP-AB,"A,A",0.011885
PTC37818.1,T6SP-AB,"A,A",0.002132
...,...,...,...
lcl|NC_003902.1_prot_NP_637297.1_1900,non-T6SP,"Y,Y",0.000000
lcl|AE014613.1_prot_AAO68872.1_1138,non-T6SP,"Y,Y",0.000000
lcl|NC_002516.2_prot_NP_249244.1_555,non-T6SP,"Y,Y",0.000000
lcl|NZ_CP010310.2_prot_WP_039404597.1_3544,non-T6SP,"Y,Y",0.000000


In [8]:
def get_star(p:float):
    if p <= 0.0001:
        return "****"
    elif p <= 0.001:
        return "***"
    elif p <= 0.01:
        return "**"
    elif p <= 0.05:
        return "*"
    else:
        return ""
from scipy.stats import wilcoxon
grouped = aac_data_transformed.groupby("Amino acid")
wilcoxon_result = {}
for name, group in grouped:
    x = group[group['Type'] == 'T6SP-AB']['DAC']
    y = group[group['Type'] == 'non-T6SP']['DAC']
    stat, p = wilcoxon(x, y)
    wilcoxon_result[name] = {
        "statistic": stat,
        "p-value": p
    }
    # print(f'Type: {name}, Wilcoxon rank-sum statistic: {stat:.2f}, p-value: {p:.3f}')
with open(f"{save_dir}/wilcoxon_result.json", "w+", encoding="UTF-8") as f:
    json.dump(wilcoxon_result, f)
aac_data_transformed.to_csv(f"{save_dir}/aac_data_transformed.csv", index_label="Seq_ID")

/home/georgezhao/.pyvirtualenvs/TxSEml_Backend/lib/python3.10/site-packages/scipy/stats/_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


处理一下，选择出显著的

In [9]:
p_values_series = pd.Series({
    k:v['p-value'] for k, v in wilcoxon_result.items()
}).to_frame().rename({0: "p-value"}, axis=1)
def swap_char(str_: str):
    str_ = list(str_)
    if str_[0] > str_[2]:
        tmp = str_[2]
        str_[2] = str_[0]
        str_[0] = tmp
    return "".join(str_)
list(set([ swap_char(item) for item in p_values_series[p_values_series['p-value'] <= 0.05].index.to_list()]))

['D,Y',
 'A,P',
 'G,Y',
 'G,K',
 'H,R',
 'S,S',
 'E,Y',
 'K,W',
 'E,K',
 'K,Y',
 'H,S',
 'F,H',
 'I,K',
 'D,E',
 'K,Q',
 'G,N',
 'E,P',
 'F,P',
 'G,Q',
 'K,R',
 'Y,Y',
 'D,K',
 'M,N',
 'V,Y',
 'K,N',
 'L,M',
 'A,H',
 'Q,V',
 'E,E',
 'G,V',
 'P,Y',
 'Q,Y',
 'A,F',
 'N,V',
 'Q,R',
 'A,Q',
 'H,L',
 'C,N',
 'I,T',
 'L,Y',
 'D,N',
 'A,L',
 'A,A',
 'E,H',
 'D,W',
 'H,I',
 'A,V',
 'A,C',
 'N,S',
 'D,T',
 'H,N',
 'L,N']